In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import json
import argparse
import pickle
import pandas as pd
import numpy as np

In [3]:
import PDB
import AnalyzeOutput as Aout
import AnalyzeMutatedSequences as Ams
import SymDesignUtils as SDUtils
import PathUtils as PUtils

In [4]:
def read_scores(file):
    """Take a json formatted score.sc file and incorporate into dictionary object

    Args:
        file (str): Location on disk of scorefile
    Returns:
        score_dict (dict): {design_name: {all_score_metric_keys: all_acore_metric_values}, ...}
    """
    with open(file, 'r') as f:
        score_dict = {}
        for score in f.readlines():
            entry = json.loads(score)
            if entry['decoy'].split('_')[-1] not in score_dict:
                score_dict[entry['decoy'].split('_')[-1]] = entry
            else:
                score_dict[entry['decoy'].split('_')[-1]].update(entry)

    return score_dict

In [5]:
def sjoin(x): 
    return ','.join(x[x.notnull()].astype(str))

In [6]:
test_dir = '/home/kylemeador/symdesign/P432/3l8r_1ho1/DEGEN_1_1/ROT_36_1/tx_4'
des_dir = SDUtils.DesignDirectory(test_dir)
scores_file = '/home/kylemeador/symdesign/P432/3l8r_1ho1/DEGEN_1_1/ROT_36_1/tx_4/scores/all_scores.sc'
# scores_file = '/home/kylemeador/symdesign/P432/3l8r_1ho1/DEGEN_1_1/ROT_36_1/tx_4/scores/all_scores_no_refine.sc'

In [7]:
all_design_scores = Aout.read_scores(scores_file)
# print(all_design_scores['refine'])
# all_design_scores = Aout.read_scores(os.path.join(des_dir.scores, scores_file))

In [75]:
scores_df = pd.DataFrame(all_design_scores).T
# scores_df = pd.DataFrame.from_dict(all_design_scores, orient='index')
display(scores_df)

,decoy,R_buns_asu,R_buns_asu_hpol,R_buns_nano,R_buns_nano_hpol,R_buns_total,R_contact_count,R_cst_weight,R_fsp_energy,R_fsp_total_stability,...,per_res_sasa_total_oligomer_271,per_res_sasa_total_oligomer_274,per_res_sasa_total_oligomer_275,per_res_sasa_total_oligomer_278,hbonds_res_selection_oligomer_B,res_type_constraint,favor_profile_switch,limit_to_profile_switch,no_constraint_switch,combo_profile_switch
refine,clean_asu_for_refine_refine,2.0,1.0,0.0,0.0,3.0,10.0,227.666977,0.00,324.735657,...,10.346614,13.316808,1.233940,0.0,"113B,169B,247B,284B,295B,330B",NaN,NaN,NaN,NaN,NaN
0001,clean_asu_for_refine_refine_design_0001,2.0,1.0,0.0,0.0,3.0,114.0,12.286269,-3.74,-697.194397,...,13.213391,23.272135,0.000000,0.0,"113B,169B,247B,284B,295B,330B",-3.74,favor_profile,NaN,NaN,NaN
0003,clean_asu_for_refine_refine_design_0003,3.0,2.0,1.0,1.0,7.0,150.0,11.413607,0.00,-692.946411,...,13.631898,21.384301,5.635777,0.0,"113B,247B,284B",NaN,NaN,limit_to_profile,NaN,NaN
0004,clean_asu_for_refine_refine_design_0004,3.0,2.0,0.0,0.0,5.0,145.0,16.463646,-3.74,-679.357239,...,13.631898,21.384301,5.635777,0.0,"247B,284B",-3.74,favor_profile,NaN,NaN,NaN
0005,clean_asu_for_refine_refine_design_0005,2.0,1.0,1.0,1.0,5.0,112.0,12.388884,-3.74,-699.148743,...,13.631898,18.182010,5.635777,0.0,"113B,247B,284B,295B",-3.74,favor_profile,NaN,NaN,NaN
0002,clean_asu_for_refine_refine_design_0002,2.0,1.0,0.0,0.0,3.0,138.0,14.589524,0.00,-697.064819,...,13.631898,23.272135,0.000000,0.0,"113B,169B,247B,284B,295B,330B",NaN,NaN,NaN,no_constraint,NaN
0006,clean_asu_for_refine_refine_design_0006,2.0,1.0,1.0,1.0,5.0,167.0,17.110184,0.00,-671.225281,...,13.631898,23.367942,0.512343,0.0,"113B,169B,247B,284B,295B,330B",NaN,NaN,NaN,NaN,combo_profile
0009,clean_asu_for_refine_refine_design_0009,2.0,1.0,0.0,0.0,3.0,125.0,11.755384,-3.74,-690.754028,...,13.213391,23.367942,0.512343,0.0,"113B,169B,247B,284B,295B,330B",-3.74,favor_profile,NaN,NaN,NaN
0007,clean_asu_for_refine_refine_design_0007,2.0,1.0,1.0,1.0,5.0,123.0,11.897225,-3.74,-696.086243,...,13.631898,23.880285,0.512343,0.0,"113B,169B,247B,284B,295B,330B",-3.74,favor_profile,NaN,NaN,NaN
0008,clean_asu_for_refine_refine_design_0008,2.0,1.0,0.0,0.0,3.0,108.0,11.959423,-3.74,-691.993652,...,13.631898,23.880285,0.512343,0.0,"113B,169B,247B,284B,295B,330B",-3.74,favor_profile,NaN,NaN,NaN


In [9]:
rename_columns = {'int_sc': 'shape_complementarity', 'int_sc_int_area': 'int_area',  # 'total_score': 'REU', 'decoy': 'design',
                  'int_sc_median_dist': 'int_separation', 'no_constraint_switch': 'protocol',
                  'limit_to_profile_switch': 'protocol', 'combo_profile_switch': 'protocol',
                  'favor_profile_switch': 'protocol', 'consensus_design_switch': 'protocol'}
remove_columns = ['decoy', 'symmetry_switch', 'oligomer_switch', 'total_score']
# remove_calc_columns = ['buns_asu_hpol', 'buns_nano_hpol', 'buns_asu', 'buns_nano', 'int_area_asu_hydrophobic', 'int_area_asu_polar', 
#                        'int_area_asu_total', 'int_area_ex_asu_hydrophobic', 'int_area_ex_asu_polar', 'int_area_ex_asu_total', 'int_connectivity1',
#                        'int_connectivity2', 'int_energy_context_asu', 'int_energy_context_unbound', 'coordinate_constraint', 'int_energy_res_summary_asu']
save_columns = ['protocol']

report_columns = {}
per_res_columns = []
hbonds_columns = []
for column in list(scores_df.columns):
    if column.startswith('R_'):
        report_columns[column] = column.replace('R_', '')
    elif column.startswith('per_res_'):
        per_res_columns.append(column)
    elif column.startswith('hbonds_res_selection'):
        hbonds_columns.append(column)

rename_columns.update(report_columns)
rename_columns.update({'R_int_sc': 'shape_complementarity'})  # remove when metrics protocol is changed
res_columns = hbonds_columns + per_res_columns
remove_columns += res_columns + save_columns

In [72]:
print(hbonds_columns)

['hbonds_res_selection_complex', 'hbonds_res_selection_oligomer_A', 'hbonds_res_selection_oligomer_B']


In [10]:
scores_df = scores_df.rename(columns=rename_columns)

In [11]:
scores_df = scores_df.groupby(level=0, axis=1).apply(lambda x: x.apply(sjoin, axis=1))
# display(scores_df)

In [12]:
strings_df = scores_df[save_columns]
# display(strings_df)
# per_res_df = scores_df[per_res_columns]

In [13]:
# np.where(scores_df.applymap(lambda x: x == ''))

In [14]:
scores_df = scores_df.replace('', np.nan)
scores_df = scores_df.drop(remove_columns, axis=1).astype(float)
# scores_df = scores_df.drop(res_columns, axis=1)

In [15]:
scores_df = scores_df.sub(scores_df.loc[PUtils.stage[1], ])

In [16]:
def sum_colums_to_new_column(df, column_dict):
    for column in column_dict:
        try:
            df[column] = df[column_dict[column][0]] + df[column_dict[column][1]]
        except KeyError:
            pass
        
    return df


def subtract_colums_to_new_column(df, column_dict):
    for column in column_dict:
        try:
            df[column] = df[column_dict[column][1]] - df[column_dict[column][0]]
        except KeyError:
            pass
        
    return df

In [17]:
# sum columns
summation_pairs = {'buns_hpol_total': ('buns_asu_hpol', 'buns_nano_hpol'),
                   'buns_heavy_total': ('buns_asu', 'buns_nano'),
                   'int_energy_context_oligomer': ('int_energy_context_A_oligomer', 'int_energy_context_B_oligomer'),
                   'int_energy_res_summary_oligomer': ('int_energy_res_summary_A_oligomer', 'int_energy_res_summary_B_oligomer')}  # ,
#                    'hbonds_oligomer': ('hbonds_res_selection_oligomer_A', 'hbonds_res_selection_oligomer_A')}

# subtract necessary delta columns using tuple [0] - [1]
delta_pairs = {'int_energy_context_delta': ('int_energy_context_complex', 'int_energy_context_oligomer'),
               'int_energy_res_summary_delta': ('int_energy_res_summary_complex', 'int_energy_res_summary_oligomer')}  # ,
#                'number_hbonds': ('hbonds_res_selection_complex', 'hbonds_oligomer')}

In [18]:
scores_df = sum_colums_to_new_column(scores_df, summation_pairs)
scores_df = subtract_colums_to_new_column(scores_df, delta_pairs)
# scores_df = scores_df.drop(remove_calc_columns, axis=1)

In [19]:
drop_unneccessary = ['int_area_asu_hydrophobic', 'int_area_asu_polar', 'int_area_asu_total', 'int_area_ex_asu_hydrophobic', 'int_area_ex_asu_polar', 'int_area_ex_asu_total',
                    'buns_asu', 'buns_asu_hpol', 'buns_nano', 'buns_nano_hpol', 'int_connectivity1', 'int_connectivity2', 'int_energy_context_asu', 'int_energy_context_unbound',
                    'coordinate_constraint', 'int_energy_res_summary_asu', 'int_energy_res_summary_unbound', 'interaction_energy', 'interaction_energy_asu', 
                    'interaction_energy_oligomerA', 'interaction_energy_oligomerB', 'interaction_energy_unbound', 'res_type_constraint', 'time', 'REU']
rosetta_terms = ['lk_ball_wtd', 'omega', 'p_aa_pp', 'pro_close', 'rama_prepro', 'yhh_planarity', 'dslf_fa13', 'fa_atr', 'fa_dun', 'fa_elec', 'fa_intra_rep',
                 'fa_intra_sol_xover4', 'fa_rep', 'fa_sol', 'hbond_bb_sc', 'hbond_lr_bb', 'hbond_sc', 'hbond_sr_bb']  # 'ref'

def drop_extras(df, drop_unneccessary):
    for unnecc in drop_unneccessary:
        try:
            df = df.drop(unnecc, axis=1)
        except KeyError:
            pass
    return df

In [20]:
scores_df = drop_extras(scores_df, drop_unneccessary)
scores_df = drop_extras(scores_df, rosetta_terms)

In [21]:
def remove_pdb_prefixes(pdb_dict):
    clean_key_dict = {}
    for key in pdb_dict:
        new_key = key.split('_')[-1]
        clean_key_dict[new_key] = pdb_dict[key]
        
    return clean_key_dict

In [22]:
wild_type_file = Ams.get_wildtype_file(des_dir)
all_mutations = Aout.design_mutations_for_metrics(des_dir, wild_type_file=wild_type_file)
all_mutations_no_chains = Ams.make_mutations_chain_agnostic(all_mutations)
all_mutations_simplified = Ams.simplify_mutation_dict(all_mutations_no_chains)
cleaned_mutations = remove_pdb_prefixes(all_mutations_simplified)

In [23]:
# from table 2 of Miller et al. 1987, sidechain is broken down into polar and non-polar
gxg_sasa = {'A': {'backbone': 46, 'polar': 0, 'non-polar': 67}, 'R': {'backbone': 45, 'polar': 107, 'non-polar': 89},
            'N': {'backbone': 45, 'polar': 69, 'non-polar': 44}, 'D': {'backbone': 45, 'polar': 58, 'non-polar': 48},
            'C': {'backbone': 36, 'polar': 69, 'non-polar': 35}, 'Q': {'backbone': 45, 'polar': 91, 'non-polar': 53},
            'E': {'backbone': 45, 'polar': 77, 'non-polar': 61}, 'G': {'backbone': 85, 'polar': 0, 'non-polar': 0},
            'H': {'backbone': 43, 'polar': 49, 'non-polar': 102}, 'I': {'backbone': 42, 'polar': 0, 'non-polar': 142},
            'L': {'backbone': 43, 'polar': 0, 'non-polar': 137}, 'K': {'backbone': 44, 'polar': 48, 'non-polar': 119},
            'M': {'backbone': 44, 'polar': 43, 'non-polar': 117}, 'F': {'backbone': 43, 'polar': 0, 'non-polar': 175},
            'P': {'backbone': 38, 'polar': 0, 'non-polar': 105}, 'S': {'backbone': 42, 'polar': 36, 'non-polar': 44},
            'T': {'backbone': 44, 'polar': 23, 'non-polar': 74}, 'Y': {'backbone': 42, 'polar': 43, 'non-polar': 144},
            'V': {'backbone': 43, 'polar': 0, 'non-polar': 117}}


def total_gxg_sasa(aa):
    s = 0
    for value in gxg_sasa[aa]:
        s += gxg_sasa[aa][value]

    return s


def sidechain_gxg_sasa(aa):
    s = 0
    for value in gxg_sasa[aa]:
        if value != 'backbone':
            s += gxg_sasa[aa][value]

    return s


def calc_relative_sasa(aa, sasa, sidechain=False):
    if sidechain:
        ref = sidechain_gxg_sasa(aa)
    else:
        ref = total_gxg_sasa(aa)
    
    return round(sasa / ref, 2)

In [80]:
def hot_spot(residue_dict, energy=-1.5):
    for res in residue_dict:
        if residue_dict[res]['energy'] <= energy:
            residue_dict[res]['hot_spot'] = 1
        else:
            residue_dict[res]['hot_spot'] = 0
    
    return residue_dict


def hbond_processing(score_dict, columns):
    """Process Hydrogen bond Metrics from Rosetta score dictionary
    
    Args:
        score_dict (dict): {'0001': {'buns': 2.0, 'per_res_energy_15': -3.26, ..., 
                            'yhh_planarity':0.885, 'hbonds_res_selection': '15A,21A,26A,35A,...'}, ...}
    Returns:
        hbond_dict (dict): {'0001': [34, 54, 67, 68, 106, 178], ...}
    """
    hbond_dict = {}
    for entry in score_dict:
        entry_dict = {}
#         for key, value in score_dict[entry].items():
#             if key.startswith('hbonds_res_selection'):
        for column in columns:
            hbonds = score_dict[entry][column].split(',')
            for i in range(len(hbonds)):
                # remove chain ID off last index
                hbonds[i] = int(hbonds[i][:-1])
            entry_dict[column.split('_')[-1]] = set(hbonds)
        if entry_dict:
            hbond_dict[entry] = list((entry_dict['complex'] - entry_dict['A']) - entry_dict['B'])
        
    return hbond_dict
    
    
def residue_processing(score_dict, mutations, hbonds=None):
    """Process Residue Metrics from Rosetta score dictionary
    
    Args:
        score_dict (dict): {'0001': {'buns': 2.0, 'per_res_energy_15': -3.26, ..., 
                            'yhh_planarity':0.885, 'hbonds_res_selection': '15A,21A,26A,35A,...'}, ...}
        mutations (dict): {'0001': {mutation_index: ('From AA', 'To AA'), ...}, ...},
    Keyword Args:
        hbonds=None (list): [34, 54, 67, 68, 106, 178]
    Returns:
        residue_dict (dict): {'0001': {15: {'aa': 'T', 'energy': -2.771, 'sasa': {'polar': 13.987, 'hydrophobic': 22.29, 'total': 36.278}, 'hbond': 0, 'core': 0, 'rim': 1, 'support': 0, 'hot_spot': 1}, ...}, ...}
    """
    dict_template = {'aa': None, 'energy': {'complex': 0, 'oligomer': 0, 'fsp': 0, 'cst': 0}, \
                     'sasa': {'polar': {'complex': 0, 'oligomer': 0}, \
                              'hydrophobic': {'complex': 0, 'oligomer': 0}, \
                              'total': {'complex': 0, 'oligomer': 0}}, \
                     'hbond': 0, 'core': 0, 'rim': 0, 'support': 0}  # , 'hot_spot': 0}
    total_residue_dict = {}
    for entry in score_dict:
        residue_dict = {}
        for key, value in score_dict[entry].items():
            if key.startswith('per_res_'):
                metadata = key.split('_')
                res = int(metadata[-1])
                if res not in residue_dict:
                    residue_dict[res] = {'aa': None, 'energy': {'complex': 0, 'oligomer': 0, 'fsp': 0, 'cst': 0}, 'sasa': {'polar': {'complex': 0, 'oligomer': 0}, 'hydrophobic': {'complex': 0, 'oligomer': 0}, 'total': {'complex': 0, 'oligomer': 0}}, 'hbond': 0, 'core': 0, 'rim': 0, 'support': 0}  # , 'hot_spot': 0}
#                     residue_dict[res] = dict_template.copy()
                r_type = metadata[2]
                pose_state = metadata[-2]
                if r_type == 'sasa':
                    polarity = metadata[3]
                    residue_dict[res][r_type][polarity][pose_state] = round(value, 3)
                else:
                    residue_dict[res][r_type][pose_state] = round(value, 3)
#             elif key.startswith('hbonds_res_selection'):
#                 hbonds = score_dict[entry][key].split(',')
#                 for i in range(len(hbonds)):
#                     # remove chain ID off last index
#                     hbonds[i] = int(hbonds[i][:-1])
#                 hbond_dict[key.split('_')[-1]] = hbonds
#         if hbond_dict:
#             num_hbonds = len(hbond_dict['complex']) - len(hbond_dict['A']) - len(hbond_dict['B'])
        if residue_dict:
            for res in residue_dict:
                try:
                    residue_dict[res]['aa'] = mutations[entry][res]
                except KeyError:
                    # fill the value with the wild_type sequence
                    residue_dict[res]['aa'] = mutations['ref'][res]
                if hbonds:
                    if res in hbonds:
                        residue_dict[res]['hbond'] = 1
                residue_dict[res]['energy'] = residue_dict[res]['energy']['complex'] - residue_dict[res]['energy']['oligomer']  # - residue_dict[res]['energy']['fsp']
#                 if residue_dict[res]['energy'] <= hot_spot_energy:
#                     residue_dict[res]['hot_spot'] = 1
                oligomer_sasa = residue_dict[res]['sasa']['total']['complex']
#                 polarities = [polarity for polarity in residue_dict[res]['sasa']]
#                 for polarity in residue_dict[res]['sasa']:
#                     residue_dict[res]['sasa_' + polarity] = residue_dict[res]['sasa'][polarity]['complex'] - residue_dict[res]['sasa'][polarity]['oligomer']
#                     polarities.append(polarity)
                for polarity in residue_dict[res]['sasa']:
                    residue_dict[res]['sasa_' + polarity] = round(residue_dict[res]['sasa'][polarity]['oligomer'] - residue_dict[res]['sasa'][polarity]['complex'], 2)
                rel_sasa = calc_relative_sasa(residue_dict[res]['aa'], residue_dict[res]['sasa_total'])
                if oligomer_sasa < 0.25:
                    residue_dict[res]['support'] = 1
                elif rel_sasa > 0.25:
                    residue_dict[res]['rim'] = 1
                else:
                    residue_dict[res]['core'] = 1
                residue_dict[res].pop('sasa')
            total_residue_dict[entry] = residue_dict
    
    return total_residue_dict

In [81]:
interface_hbonds = hbond_processing(all_design_scores, hbonds_columns)
residue_dict = residue_processing(all_design_scores, cleaned_mutations, hbonds=interface_hbonds)

In [26]:
# all_design_sequences = Ams.mutate_wildtype_sequences(sequence_dir, wild_type_file)
sequence_mutations = Aout.design_mutations_for_sequence(des_dir, wild_type_file)
sequence_mutations.pop('ref')
wt_sequence = Ams.get_pdb_sequences(wild_type_file)
all_design_sequences = Ams.generate_sequences(wt_sequence, sequence_mutations)

# Rename all sequences to only nstruct index
for chain in all_design_sequences:
    all_design_sequences[chain] = remove_pdb_prefixes(all_design_sequences[chain])

In [27]:
# unique_protocols = scores_df.columns.get_level_values(1)=='protocol'  
unique_protocols = strings_df['protocol'].unique().tolist()  # ['favor_profile', 'limit_to_profile', 'no_constraint', 'combo_profile']
# will need to drop the refine and consensus protocols or add value
unique_protocols.remove('')
protocol_specific_designs = {}
for protocol in unique_protocols:
    protocol_specific_designs[protocol] = strings_df.index[strings_df['protocol'] == protocol].tolist()

In [28]:
protocol_specific_sequences = {}
for protocol in protocol_specific_designs:
    protocol_specific_sequences[protocol] = {chain: {name: all_design_sequences[chain][name] 
                                                     for name in all_design_sequences[chain] 
                                                     if name in protocol_specific_designs[protocol]}
                                             for chain in all_design_sequences}
# sequence_analysis = Ams.analyze_mutations(des_dir, all_design_sequences)  # , print_results=print_output

In [29]:
def per_res_metric(divergence_dict, key='jsd'):
    """Find Metric Value/Residue specified by key
    
    Args:
        divergence_dict (dict): {16: {'S': 0.134, 'A': 0.050, ..., 'jsd': 0.732, 'int_jsd': 0.412}, ...}
    Keyword Args:
        key='jsd' (str): Name of the residue metric to average
    Returns:
        jsd_per_res (float): 0.367
    """
    s = 0.0
    for residue in divergence_dict:
        s += divergence_dict[residue][key]
    
    return round(s / len(divergence_dict), 3)

In [30]:
per_res_keys = ['jsd', 'int_jsd']
protocol_specific_stats = {}
for protocol in protocol_specific_sequences:
    res_profile = Ams.analyze_mutations(des_dir, protocol_specific_sequences[protocol])
#     protocol_specific_stats[protocol] = {'res_profile': Ams.analyze_mutations(des_dir, protocol_specific_sequences[protocol])}  # , print_results=print_output
    protocol_specific_stats[protocol] = {}
    for key in per_res_keys:
        protocol_specific_stats[protocol][key + '_per_res'] = per_res_metric(res_profile, key=key)
#         protocol_specific_stats[protocol][key + '_per_res'] = per_res_metric(protocol_specific_stats[protocol]['res_profile'], key=key)
        
    # finally get rid of per residue stats from protocol_specific_stats
#     protocol_specific_stats[protocol].pop('res_profile')
#         protocol_specific_stats[protocol]['int_jsd_per_res'] = per_res_metric(protocol_specific_stats[protocol]['res_profile'], key='int_jsd')
# {protocol: {'res_profile': {16: {'S': 0.134, 'A': 0.050, ..., 'jsd': 0.732, 'int_jsd': 0.412}, 'jsd_per_res': 0.747, 'int_jsd_per_res': 0.412}, ...}, ...}

In [82]:
# Create extra df objects
for entry in interface_hbonds:
    interface_hbonds[entry] = len(interface_hbonds[entry])

In [83]:
number_hbonds_df = pd.DataFrame(interface_hbonds, index=['number_hbonds', ]).T  # from_dict(interface_hbonds, orient='index', columns=['number_hbonds',])

,number_hbonds
refine,3
0001,17
0003,11
0004,15
0005,14
0002,14
0006,14
0009,15
0007,14
0008,14


In [31]:
scores_df = pd.merge(scores_df, number_hbonds_df, left_index=True, right_index=True)

In [32]:
# print(cleaned_mutations['ref'])
# print(residue_dict['refine'][15])

In [33]:
res_df = pd.concat({k: pd.DataFrame(v) for k, v in residue_dict.items()}).unstack()

In [34]:
# subtract residue info from reference (refine), currently only subtracting energy. Also refine is not perfect reference in deltaG as modelling occurred
delta_g_res = False
if delta_g_res:
    res_df.update(res_df.iloc[:, res_df.columns.get_level_values(1)=='energy'].sub(res_df.loc[PUtils.stage[1], res_df.columns.get_level_values(1)=='energy']))  # [res_columns_subtract]
    # df = df.sub(df.loc[PUtils.stage[1], df.columns.get_level_values(1)=='energy'])  # [res_columns_subtract]
    display(res_df)

In [35]:
strings_df.columns = pd.MultiIndex.from_product([['protocol'], strings_df.columns])
scores_df.columns = pd.MultiIndex.from_product([['pose'], scores_df.columns])

In [36]:
res_df = pd.merge(strings_df, res_df, left_index=True, right_index=True)

In [37]:
scores_df = pd.merge(strings_df, scores_df, left_index=True, right_index=True)

In [38]:
merge_residue_data = False
if merge_residue_data:
    scores_df = pd.merge(scores_df, res_df, left_index=True, right_index=True)  #, how='inner', on='index',

In [39]:
# drop refine row
scores_df = scores_df.drop(PUtils.stage[1], axis=0)

In [40]:
# scores_df = pd.merge(scores_df, df, left_index=True, right_index=True)

In [41]:
scores_df = scores_df.append(scores_df.mean().rename('average'))
scores_df = scores_df.append(scores_df.std().rename('std_dev'))

In [42]:
# # unique_protocols = scores_df['protocol'].unique()  # ['favor_profile', 'limit_to_profile', 'no_constraint', 'combo_profile']
# # unique_protocols = scores_df.columns.get_level_values(1)=='protocol'  # ['favor_profile', 'limit_to_profile', 'no_constraint', 'combo_profile']
# # print(unique_protocols)
# # display(scores_df.loc[:, scores_df.columns.get_level_values(1)=='protocol'])
# # temp = scores_df.loc[:, scores_df.columns.get_level_values(1)=='protocol']
# # display(temp['protocol'] == protocol)
# # display(scores_df.xs(('protocol', 'protocol'), level=(1, 0), axis=1))  #.mean())
# display(scores_df.xs(('protocol', 'protocol'), level=(1, 0), axis=1) == 'favor_profile')  #.mean())
# # display(scores_df.loc[scores_df.xs(('protocol', 'protocol'), level=(1, 0), axis=1) == 'favor_profile', :])  #.mean())
# display(scores_df.groupby(('protocol', 'protocol')).mean())  # , level=1, axis=1).mean())

In [43]:
# for protocol in unique_protocols:
#     temp_df = scores_df.loc[:, scores_df.columns.get_level_values(1)=='protocol']
#     scores_df = scores_df.append(scores_df.loc[temp_df['protocol'] == protocol, :].mean().rename(str(protocol)))  # + '_average'))
#     scores_df = scores_df.append(scores_df[scores_df['protocol'] == protocol].mean().rename(str(protocol)))  # + '_average'))
scores_df = scores_df.append(scores_df.groupby(('protocol', 'protocol')).mean())  #.rename(str(protocol)))
std_df = scores_df.groupby(('protocol', 'protocol')).std()  # .index.to_series().map({protocol: protocol + '_std_dev' for protocol in sorted(unique_protocols)})
std_df.index = std_df.index.to_series().map({protocol: protocol + '_std_dev' for protocol in sorted(unique_protocols)})
scores_df = scores_df.append(std_df)  #.rename(str(protocol)))
# std_df = scores_df.groupby(('protocol', 'protocol')).std().index.rename(rename_dev)
# scores_df = scores_df.append(scores_df.groupby(('protocol', 'protocol')).std())  #.rename(str(protocol)))

#     scores_df = scores_df.append(scores_df[scores_df.xs(('protocol', 'protocol'), level=(1, 0), axis=1) == 'favor_profile'].mean().rename(str(protocol)))  # + '_average'))
#     scores_df = scores_df.append(scores_df[scores_df.xs(('protocol', 'protocol'), level=(1, 0), axis=1) == 'favor_profile'].std().rename(str(protocol) + '_std_dev'))
#     scores_df = scores_df.append(scores_df[scores_df.loc[:, scores_df.columns.get_level_values(1)=='protocol'] == protocol].std().rename(str(protocol) + '_std_dev'))    
#     scores_df = scores_df.append(scores_df[scores_df['protocol'] == protocol].std().rename(str(protocol) + '_std_dev'))

In [44]:
display(scores_df)

pose                             \
                         buns_heavy_total buns_hpol_total buns_total   
0001                             0.000000        0.000000   0.000000   
0003                             2.000000        2.000000   4.000000   
0004                             1.000000        1.000000   2.000000   
0005                             1.000000        1.000000   2.000000   
0002                             0.000000        0.000000   0.000000   
0006                             1.000000        1.000000   2.000000   
0009                             0.000000        0.000000   0.000000   
0007                             1.000000        1.000000   2.000000   
0008                             0.000000        0.000000   0.000000   
0010                             0.000000        0.000000   0.000000   
average                          0.600000        0.600000   1.200000   
std_dev                          0.663325        0.663325   1.326650   
combo_profile                    1.000000        1.000000   2.000000   
favor_profile                    0.500000        0.500000   1.000000   
limit_to_profile                 1.000000        1.000000   2.000000   
no_constraint                    0.000000        0.000000   0.000000   
combo_profile_std_dev                 NaN             NaN        NaN   
favor_profile_std_dev            0.547723        0.547723   1.095445   
limit_to_profile_std_dev         1.414214        1.414214   2.828427   
no_constraint_std_dev                 NaN             NaN        NaN   

                                                               \
                         contact_count  cst_weight fsp_energy   
0001                        104.000000 -215.380708  -3.740000   
0003                        140.000000 -216.253370   0.000000   
0004                        135.000000 -211.203331  -3.740000   
0005                        102.000000 -215.278093  -3.740000   
0002                        128.000000 -213.077453   0.000000   
0006                        157.000000 -210.556793   0.000000   
0009                        115.000000 -215.911592  -3.740000   
0007                        113.000000 -215.769752  -3.740000   
0008                         98.000000 -215.707554  -3.740000   
0010                        160.000000 -208.190561   0.000000   
average                     125.200000 -213.732921  -2.244000   
std_dev                      21.273458    2.681117   1.832218   
combo_profile               157.000000 -210.556793   0.000000   
favor_profile               111.166667 -214.875172  -3.740000   
limit_to_profile            150.000000 -212.221966   0.000000   
no_constraint               128.000000 -213.077453   0.000000   
combo_profile_std_dev              NaN         NaN        NaN   
favor_profile_std_dev        13.377842    1.814807   0.000000   
limit_to_profile_std_dev     14.142136    5.701267   0.000000   
no_constraint_std_dev              NaN         NaN        NaN   

                                                             \
                         fsp_total_stability full_stability   
0001                            -1021.930054   -1018.190063   
0003                            -1017.682068   -1017.682068   
0004                            -1004.092896   -1000.352905   
0005                            -1023.884399   -1020.144409   
0002                            -1021.800476   -1021.800476   
0006                             -995.960938    -995.960938   
0009                            -1015.489685   -1011.749695   
0007                            -1020.821899   -1017.081909   
0008                            -1016.729309   -1012.989319   
0010                             -990.061096    -990.061096   
average                         -1012.845282   -1010.601288   
std_dev                            11.289352      10.557029   
combo_profile                    -995.960938    -995.960938   
favor_profile                   -1017.158040   -1013.418050   
limit_to_prof

In [45]:
# Create pose_average_df
pose_average_df = pd.DataFrame(scores_df.loc['average', :]).T
pose_average_df.index = [str(des_dir),]
# pose_average_df.columns = pd.MultiIndex.from_product([['average'], pose_average_df.columns])

In [46]:
def gather_fragment_metrics(_des_dir):
    with open(os.path.join(_des_dir.path, PUtils.frag_file), 'r') as f:
        frag_match_info_file = f.readlines()
        residue_cluster_dict = {}
        for line in frag_match_info_file:
            if line[:12] == 'Cluster ID: ':
                cluster = line[12:].split()[0].strip().replace('i', '').replace('j', '').replace('k', '')
            if line[:17] == 'Overlap Z-Value: ':
                residue_cluster_dict[cluster] = float(line[17:].strip())
            if line[:17] == 'Nanohedra Score: ':
                nanohedra_score = float(line[17:].strip())
#             if line[:39] == 'Unique Interface Fragment Match Count: ':
#                 int_match = int(line[39:].strip())
#             if line[:39] == 'Unique Interface Fragment Total Count: ':
#                 int_total = int(line[39:].strip())
        fragment_z_total = 0
        for cluster in residue_cluster_dict:
            fragment_z_total += residue_cluster_dict[cluster]
        num_fragments = len(residue_cluster_dict)
        ave_z = fragment_z_total / num_fragments
                
    return {'nanohedra_score': nanohedra_score, 'average_fragment_z_score': ave_z, 'unique_fragments': num_fragments}  # , 'int_total': int_total}

In [47]:
# Remove pose specific metrics from pose_average_df
remove_average_columns = ['protocol',]
pose_average_df = pose_average_df.drop(remove_average_columns, level=1, axis=1)

In [48]:
# Add pose specific metrics to pose_average_df
other_pose_metrics = gather_fragment_metrics(des_dir)
int_residues = SDUtils.get_interface_residues(SDUtils.parse_design_flags(des_dir.path))
other_pose_metrics['total_interface_residues'] = len(int_residues)
wt_pdb = SDUtils.read_pdb(wild_type_file)
# this only works with two chains TODO
chain_sep = wt_pdb.getTermCAAtom('C', wt_pdb.chain_id_list[0]).residue_number
int_b_factor = 0
for residue in int_residues:
    if residue <= chain_sep:
        int_b_factor += wt_pdb.get_ave_residue_b_factor(wt_pdb.chain_id_list[0], residue)
    else:
        int_b_factor += wt_pdb.get_ave_residue_b_factor(wt_pdb.chain_id_list[1], residue)
        
other_pose_metrics['interface_b_factor_per_res'] = round(int_b_factor / len(int_residues), 2)

In [49]:
fragment_metrics_df = pd.DataFrame(other_pose_metrics, index=[str(des_dir),])  # .from_dict(orient='index') index=[str(des_dir),])  # , columns=[str(des_dir),])  # orient='columns', 
# fragment_metrics_df.index = [str(des_dir),]
# fragment_metrics_df = fragment_metrics_df.T
fragment_metrics_df.columns = pd.MultiIndex.from_product([['pose'], fragment_metrics_df.columns])
# fragment_metrics_df.columns = pd.MultiIndex.from_product([['fragment'], pose_average_df.columns])
pose_average_df = pd.merge(pose_average_df, fragment_metrics_df, left_index=True, right_index=True)

In [60]:
# add Protocol specific metrics to total_average
columns_to_add = ['full_stability', 'int_energy_res_summary_delta', 'int_energy_context_delta', 'shape_complementarity', 
                  'buns_total', 'contact_count', 'interaction_energy_complex', 'int_area_hydrophobic', 'int_area_polar',
                  'int_area_total', 'shape_complementarity', 'number_hbonds']
for protocol in unique_protocols:
#     for column in columns_to_add:
    temp_df = pd.DataFrame(scores_df.loc[protocol, (slice(None), columns_to_add)]).T
    # from protocol dependent sequence statistics add stats as columns
    # {protocol: {'jsd_per_res': 0.747, 'int_jsd_per_res': 0.412}, ...}, ...}  # 'res_profile': {16: {'S': 0.134, 'A': 0.050, ..., 'jsd': 0.732, 'int_jsd': 0.412},
    for stat in protocol_specific_stats[protocol]:
        temp_df[('', stat)] = protocol_specific_stats[protocol][stat]

    temp_df.index = [str(des_dir), ]
    temp_df.columns = temp_df.columns.droplevel(0)    
    temp_df.columns = pd.MultiIndex.from_product([[protocol], temp_df.columns])
    pose_average_df = pd.merge(pose_average_df, temp_df, left_index=True, right_index=True)

pose                                                    \
              buns_total contact_count full_stability int_area_hydrophobic   
favor_profile          1       111.167       -1013.42              244.909   

                                                                      \
              int_area_polar int_area_total int_energy_context_delta   
favor_profile        297.551        542.459                 -8.16781   

                                                                       \
              int_energy_res_summary_delta interaction_energy_complex   
favor_profile                     -8.49447                   -15.9626   

                                                   
              number_hbonds shape_complementarity  
favor_profile       14.8333             0.0602047

pose                                                    \
                 buns_total contact_count full_stability int_area_hydrophobic   
limit_to_profile          2           150       -1003.87              257.114   

                                                                         \
                 int_area_polar int_area_total int_energy_context_delta   
limit_to_profile        270.897        528.011                 -10.4729   

                                                                          \
                 int_energy_res_summary_delta interaction_energy_complex   
limit_to_profile                      -11.068                   -15.1796   

                                                      
                 number_hbonds shape_complementarity  
limit_to_profile            14             0.0383505

pose                                                    \
              buns_total contact_count full_stability int_area_hydrophobic   
no_constraint          0           128        -1021.8              340.765   

                                                                      \
              int_area_polar int_area_total int_energy_context_delta   
no_constraint        241.905         582.67                 -28.8389   

                                                                       \
              int_energy_res_summary_delta interaction_energy_complex   
no_constraint                       -29.99                   -17.3215   

                                                   
              number_hbonds shape_complementarity  
no_constraint            14              0.118689

pose                                                    \
              buns_total contact_count full_stability int_area_hydrophobic   
combo_profile          2           157       -995.961              236.802   

                                                                      \
              int_area_polar int_area_total int_energy_context_delta   
combo_profile        373.381        610.183                 -34.9458   

                                                                       \
              int_energy_res_summary_delta interaction_energy_complex   
combo_profile                     -35.4579                   -14.5771   

                                                   
              number_hbonds shape_complementarity  
combo_profile            14              0.166845

In [ ]:
display(pose_average_df)

In [ ]:
pose_average_df.to_csv('pose_average_metrics_df.csv')